# Gasoline consumption

In [1]:
import pandas as pd
from linearmodels import PooledOLS # Pooled model
from linearmodels import PanelOLS # Fixed-effect model
from linearmodels import RandomEffects # Random-effect model
from linearmodels import IVGMM # GMM-method
from linearmodels.panel import compare # сравнение моделей

In [2]:
gasoline_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/panel-analysis/panels-csv/Gasoline.csv')
gasoline_df.head()

,country,year,lgaspcar,lincomep,lrpmg,lcarpcap
0,AUSTRIA,1960,4.173244,-6.474277,-0.334548,-9.766840
1,AUSTRIA,1961,4.100989,-6.426006,-0.351328,-9.608622
2,AUSTRIA,1962,4.073177,-6.407308,-0.379518,-9.457257
3,AUSTRIA,1963,4.059509,-6.370679,-0.414251,-9.343155
4,AUSTRIA,1964,4.037689,-6.322247,-0.445335,-9.237739


In [3]:
gasoline_panel = gasoline_df.set_index(['country', 'year'])
gasoline_panel

lgaspcar  lincomep     lrpmg  lcarpcap
country year                                        
AUSTRIA 1960  4.173244 -6.474277 -0.334548 -9.766840
        1961  4.100989 -6.426006 -0.351328 -9.608622
        1962  4.073177 -6.407308 -0.379518 -9.457257
        1963  4.059509 -6.370679 -0.414251 -9.343155
        1964  4.037689 -6.322247 -0.445335 -9.237739
...                ...       ...       ...       ...
U.S.A.  1974  4.798626 -5.328694 -1.231467 -7.617558
        1975  4.804932 -5.346190 -1.200377 -7.607010
        1976  4.814891 -5.297946 -1.154682 -7.574748
        1977  4.811032 -5.256606 -1.175910 -7.553458
        1978  4.818454 -5.221232 -1.212062 -7.536176

[342 rows x 4 columns]

In [5]:
# Pool, RE, FE (non-robust s.e.)
pl = PooledOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=gasoline_panel).fit()
re = RandomEffects.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=gasoline_panel).fit()
fe = PanelOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap+EntityEffects', data=gasoline_panel).fit()

In [6]:
compare({'Pool': pl, 'RE': re, 'FE': fe}, stars=True)

,Pool,RE,FE
Dep. Variable,lgaspcar,lgaspcar,lgaspcar
Estimator,PooledOLS,RandomEffects,PanelOLS
No. Observations,342,342,342
Cov. Est.,Unadjusted,Unadjusted,Unadjusted
R-squared,0.8549,0.8293,0.8396
R-Squared (Within),0.7435,0.8363,0.8396
R-Squared (Between),0.8771,0.7099,0.5424
R-Squared (Overall),0.8549,0.7309,0.5917
F-statistic,664.00,547.40,560.09
P-value (F-stat),0.0000,0.0000,0.0000


In [7]:
# Pool, RE, FE (robust s.e., AB)
pl = PooledOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=gasoline_panel).fit(cov_type='clustered', cluster_entity=True)
re = RandomEffects.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap', data=gasoline_panel).fit(cov_type='clustered', cluster_entity=True)
fe = PanelOLS.from_formula(formula='lgaspcar~1+lincomep+lrpmg+lcarpcap+EntityEffects', data=gasoline_panel).fit(cov_type='clustered', cluster_entity=True)
compare({'Pool': pl, 'RE': re, 'FE': fe}, stars=True)

,Pool,RE,FE
Dep. Variable,lgaspcar,lgaspcar,lgaspcar
Estimator,PooledOLS,RandomEffects,PanelOLS
No. Observations,342,342,342
Cov. Est.,Clustered,Clustered,Clustered
R-squared,0.8549,0.8293,0.8396
R-Squared (Within),0.7435,0.8363,0.8396
R-Squared (Between),0.8771,0.7099,0.5424
R-Squared (Overall),0.8549,0.7309,0.5917
F-statistic,664.00,547.40,560.09
P-value (F-stat),0.0000,0.0000,0.0000


# Тест Хаусмана

In [8]:
import numpy as np
from scipy.stats import chi2

In [9]:
(fe.params-re.params).T@np.linalg.inv(fe.cov-re.cov)@(fe.params-re.params)

184.0936906703273

In [10]:
chi2.ppf(q=1-0.05, df=3)

7.814727903251179